Similar to replication, but instead of doing LASSO subset selection on whole period and then using those subsets for entire backtest, do LASSO subset selection at each step. If no predictors selected for an industry, use all predictors.

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time 
import copy
import random

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy Numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.regularizers import L1L2
from keras.models import Sequential
from keras.models import load_model

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


Using TensorFlow backend.


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']
    data[ind+".lead"] = data[ind].shift(-1)

data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')
    
data


Loading data...


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
192609,0.93,3.79,1.03,6.35,-1.22,0.50,-0.74,0.46,5.10,2.08,...,-0.43,-2.32,-1.41,-5.40,-2.96,-15.70,-2.52,-4.43,-5.48,-8.81
192610,-3.38,-3.63,0.74,-5.08,9.15,-5.00,-0.20,-0.89,-5.08,0.68,...,1.32,3.46,3.33,3.53,1.29,4.36,6.21,4.02,1.93,3.69
192611,6.04,6.98,4.24,1.35,-6.11,-0.85,1.56,5.11,4.89,2.80,...,1.71,5.93,6.96,-4.91,3.31,9.37,0.29,1.23,2.40,-2.62
192612,-0.79,-4.37,2.27,1.89,0.25,2.28,0.32,-0.17,5.09,6.15,...,1.63,1.83,-1.70,-2.85,1.19,-18.18,-2.95,1.27,-2.63,-4.21
192701,-1.09,0.32,-0.60,1.62,9.42,3.83,-1.31,4.80,-0.02,-1.75,...,3.71,8.64,4.59,5.20,4.92,3.23,3.21,6.60,2.71,0.02
192702,4.16,12.57,1.23,1.38,1.15,4.31,2.51,1.45,9.43,7.48,...,5.26,-8.10,3.99,-9.97,0.76,-20.77,-0.60,-2.72,1.06,1.88
192703,1.79,-13.86,5.21,0.72,-0.58,-0.38,0.60,0.71,5.77,-5.69,...,-2.38,3.19,2.85,4.68,0.49,-11.00,4.13,6.31,2.64,5.02
192704,2.47,2.60,3.76,-4.02,-1.21,1.19,5.70,2.49,2.70,0.29,...,3.05,18.03,4.80,5.06,6.62,-4.31,1.94,4.32,9.95,1.34
192705,5.83,11.32,11.50,-0.12,2.83,10.24,3.07,3.82,3.55,12.92,...,-2.40,-7.06,2.47,5.18,-2.23,-4.44,-0.96,1.01,-0.29,-4.08


In [3]:
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,...,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000
mean,0.688666,0.727030,0.985079,0.732095,0.532253,0.564333,0.690387,0.665825,0.552367,0.687145,...,0.515968,0.729928,0.622970,0.534806,0.601090,0.631076,0.698235,0.728766,0.637547,0.396628
std,4.308660,5.058992,6.032324,7.128170,5.780362,4.728000,6.355251,4.897557,5.482363,6.970961,...,4.607931,6.486956,6.698787,5.021876,5.707154,5.571040,5.334178,6.065564,5.381389,5.771655
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.630000,-2.080000,-2.740000,-3.390000,-2.600000,-2.030000,-2.800000,-2.230000,-2.750000,-3.170000,...,-2.110000,-3.050000,-3.220000,-2.400000,-2.780000,-2.560000,-2.380000,-2.840000,-2.400000,-2.930000
50%,0.740000,0.750000,1.270000,0.940000,0.510000,0.750000,0.700000,0.760000,0.720000,0.640000,...,0.590000,1.010000,0.670000,0.710000,0.900000,0.940000,0.540000,1.080000,0.870000,0.540000
75%,3.070000,3.690000,4.660000,5.260000,3.640000,3.540000,4.310000,3.550000,3.760000,4.480000,...,3.360000,4.260000,4.630000,3.460000,4.040000,3.880000,3.980000,4.300000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [5]:
# annualized returns don't match Table 1, oddly
# geometric mean, annualized
pd.DataFrame((np.prod(data/100 + 1)**(12.0/len(data))-1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.073929
Beer,0.074309
Smoke,0.100741
Games,0.058342
Books,0.044662
Hshld,0.055568
Clths,0.060067
Hlth,0.067552
Chems,0.049242
Txtls,0.054817


In [6]:
# try this way, arithmetic mean then annualize (not very correct)
#print(pd.DataFrame(((desc.loc['mean']/100+1)**12-1)[:30]))
#nope

# same
pd.DataFrame(((1 + np.mean(data, axis=0)/100)**12 -1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.085843
Beer,0.090818
Smoke,0.124829
Games,0.091476
Books,0.065774
Hshld,0.069862
Clths,0.086066
Hlth,0.082891
Chems,0.068335
Txtls,0.085646


In [7]:
#annualized volatility 
pd.DataFrame((desc.loc['std']*np.sqrt(12))[:30].round(2))
# lines up with table 1

,std
Food,14.93
Beer,17.52
Smoke,20.90
Games,24.69
Books,20.02
Hshld,16.38
Clths,22.02
Hlth,16.97
Chems,18.99
Txtls,24.15


In [8]:
# Run LASSO, then OLS on selected variables

# skip last row to better match published r-squared
# looks like they forecast actuals 1960-2016 using 1959m12 to 2016m11
# not exact matches to Table 2 R-squared but almost within rounding error 
X = data.values[:-1,:30]
Y = data.values[:-1,-30:]
nrows = X.shape[0]
coef_dict = {}
X.shape

(696, 30)

In [9]:
def subset_selection(X, Y, model_aic, verbose=False):
    
    global industries
    coef_dict = {}

    for ind in industries:
        y = Y[:,ind_reverse_dict[ind]]
        
        model_aic.fit(X, y)
        
        coef_dict[ind] = [indstr for i, indstr in enumerate(industries) if model_aic.coef_[i] !=0]
        #if verbose:
        #    print("LASSO variables selected for %s: " % ind)
        # print(coef_dict[ind])
        #y_pred = model_aic.predict(X)
        # print ("In-sample LASSO R-squared: %.6f" % r2_score(y, y_pred))
        if not coef_dict[ind]:
            coef_dict[ind] = industries
        # fit OLS vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        if verbose:
            # col nums of selected predictors
            indcols = [ind_reverse_dict[indstr] for indstr in coef_dict[ind]]
            model_ols = LinearRegression()
            model_ols.fit(X[:, indcols], y)
            y_pred = model_ols.predict(X[:, indcols])
            print ("In-sample OLS R-squared for %s against %s : %.4f" % (str(ind), str(coef_dict[ind]), r2_score(y, y_pred)))
            #print("---")
            
    return coef_dict

coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'))

# These subsets line up closely with Table 2
# except Clths, Whlsl, we get different predictors

In [10]:
def predict_with_subsets(X, Y, model):

    global industries
    
    scores = []
    coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'))
    
    for ind in industries:
        y = Y[:,ind_reverse_dict[ind]]

#        print("LASSO variables selected for %s: " % ind)
#        print(coef_dict[ind])
        
        if not coef_dict[ind]:
            print("No coefs selected for " + ind)
 #           print("---")
            continue
        # fit model vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        #print("Running model for " + ind + " against " + str(coef_dict[ind]))
        # col nums of selected predictors
        indcols = [ind_reverse_dict[indstr] for indstr in coef_dict[ind]]
        model.fit(X[:, indcols], y)
        y_pred = model.predict(X[:, indcols])
        score = r2_score(y, y_pred)
        scores.append(score)
        print ("In-sample R-squared: %.4f for %s against %s" % (score, ind, str(coef_dict[ind])))
#        print("---")
    
    print("Mean R-squared: %.4f" % np.mean(np.array(scores)))
    

predict_with_subsets(X, Y, LinearRegression())


In-sample R-squared: 0.0225 for Food against ['Clths', 'Coal', 'Util', 'Rtail']
In-sample R-squared: 0.0255 for Beer against ['Food', 'Clths', 'Coal']
In-sample R-squared: 0.0660 for Smoke against ['Hshld', 'Txtls', 'ElcEq', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
In-sample R-squared: 0.0507 for Games against ['Books', 'Clths', 'Coal', 'Fin']
In-sample R-squared: 0.0631 for Books against ['Games', 'Books', 'Clths', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Meals', 'Fin']
In-sample R-squared: 0.0301 for Hshld against ['Clths', 'Coal', 'Rtail']
In-sample R-squared: 0.0442 for Clths against ['Clths', 'Coal', 'Servs', 'Rtail']
In-sample R-squared: 0.0285 for Hlth against ['Books', 'Mines', 'Coal', 'Oil', 'Util']
In-sample R-squared: 0.0075 for Chems against ['Clths']
In-sample R-squared: 0.0787 for Txtls against ['Clths', 'Autos', 'Coal', 'Oil', 'Rtail', 'Fin']
In-sample R-squared: 0.0519 for Cnstr against ['Clths', 'Coal', 'Oil', 'Util'

In [11]:
def fit_predict(X, Y, model):
    """for backtest, fit Ys v. X using n-1 rows
    predict Ys on X using nth row
    return a prediction for month n+1 using X for final month"""
    
    global industries
    
    # keep last row to predict against
    X_predict = X[-1]
    X_predict = X_predict.reshape(1,X.shape[1])
    # fit on remaining rows
    X_fit = X[:-1]
    Y_fit = Y[:-1]

    predictions = []
    # LASSO subset selection 
    coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'))
    
    for ind in industries:
        # column indexes to fit against each other
        indcols = [ind_reverse_dict[indstr] for indstr in coef_dict[ind]]
        responsecol = ind_reverse_dict[ind]
        model.fit(X_fit[:, indcols], Y_fit[:,responsecol])
        y_pred = model.predict(X_predict[:,indcols])        
        predictions.append(y_pred[0])
        
    return predictions

#    return np.argsort(predictions)

X = data.values[:,:30]
Y = data.values[:, -30:]
model = LinearRegression()
predictions = fit_predict(X, Y, model)
predictions

[-0.22043631464109348,
 0.18641200394329466,
 -1.3283611504933985,
 0.2731610731305569,
 -1.3535794150375275,
 0.4623685212518487,
 0.3863796304349979,
 -1.201615972466105,
 0.8441317988995884,
 -0.4390484419649552,
 -1.2412946182920859,
 0.35380818707447625,
 0.7504367303713203,
 -0.94563328689501,
 -1.7528041552648477,
 1.123323563478705,
 -3.6273515502846414,
 -0.6446073935669205,
 0.9716840348987886,
 -1.224636810694185,
 -1.495351956318207,
 -2.1555323195588016,
 -0.9977699875391348,
 -0.06654426629301385,
 -3.814789955934355,
 -2.3048178216647632,
 0.9092061512631813,
 -2.362881624684105,
 0.6624203768057081,
 -1.6243332625709201]

In [12]:
# 197001 = 121
STARTMONTH = 121
print(X[STARTMONTH])
print(data.iloc[STARTMONTH][:30])

[ -3.34  -1.95  -7.59  -7.76 -12.05  -7.5   -5.69  -7.71  -7.37  -5.26
  -9.84  -6.31  -7.15  -6.89  -9.35 -12.49  -2.34  -0.77 -12.16  -4.83
  -3.16 -11.17  -9.73  -8.89  -8.17  -8.28  -6.31 -13.12  -9.78  -6.2 ]
Food     -3.34
Beer     -1.95
Smoke    -7.59
Games    -7.76
Books   -12.05
Hshld    -7.50
Clths    -5.69
Hlth     -7.71
Chems    -7.37
Txtls    -5.26
Cnstr    -9.84
Steel    -6.31
FabPr    -7.15
ElcEq    -6.89
Autos    -9.35
Carry   -12.49
Mines    -2.34
Coal     -0.77
Oil     -12.16
Util     -4.83
Telcm    -3.16
Servs   -11.17
BusEq    -9.73
Paper    -8.89
Trans    -8.17
Whlsl    -8.28
Rtail    -6.31
Meals   -13.12
Fin      -9.78
Other    -6.20
Name: 197001, dtype: float64


In [13]:
# predict all months starting STARTMONTH
# initialize predictions matrix P

def run_backtest(X, Y, estimator, startmonth=0):
    global P
    global R 

    P = np.zeros_like(X)
    count = 0
    for month_index in range(startmonth, X.shape[0]+1):
        predictions = fit_predict(X[:month_index, :], 
                                  Y[:month_index], 
                                  estimator,)
        try:
            P[month_index]= predictions
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
            sys.stdout.flush()
        except IndexError:
            # I want to run the fit and see the R-squared on full dataset
            # but we are storing the predictions in row of the month predicted
            # so we have no row to store the last prediction (2017-01)
            print("\nlast prediction not stored")
                
    R = np.zeros(P.shape[0])
    numstocks = 6 # top quintile (and bottom)

    for month_index in range(startmonth, X.shape[0]):
        # get indexes of sorted smallest to largest
        select_array = np.argsort(P[month_index])
        # leftmost 6
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        R[month_index] = np.mean(X[month_index, long_indexes])/2 - np.mean(X[month_index, short_indexes])/2

    results = R[startmonth:]

    index = pd.date_range('01/01/1970',periods=results.shape[0], freq='M')
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

    stats = perfdata['Equity'].calc_stats()

    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])   
    return retframe


In [14]:
model = LinearRegression()
run_backtest(X, Y, model, startmonth=STARTMONTH)

................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................................................................................
................
last prediction not stored


,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0342349
yearly_vol,0.0479222
yearly_sharpe,0.731
max_drawdown,-0.128334
sortino,0.317025


In [15]:
# double check results
#model = LinearRegression()
#R = run_backtest(X, Y, model, coef_dict_paper, startmonth=STARTMONTH, summary=False)
results = R[STARTMONTH:]
print(len(results))
#print(results)
print(np.mean(results))
print(np.std(results) * np.sqrt(12))
print(np.prod(1 + results / 100))
print(np.prod(1 + results / 100) ** (12.0/results.shape[0]))-1

576
0.29151765046296296
5.4715438056558225
4.981156109052597
0.03401707756367234


In [16]:
# calc MSE across all preds
np.mean((P[121:]-X[121:])**2)

40.98430668761095

In [17]:
# run chart
perf = 100 * np.cumprod(1 + results / 100)

x_coords = np.linspace(1970, 2016, perf.shape[0])

trace1 = Scatter(
    x = x_coords,
    y = perf,
    name = 'Growth of $1',    
)

layout = Layout(
    yaxis=dict(
        type='log',
        autorange=True
    )
)
plotdata = [trace1]

fig = Figure(data=plotdata, layout=layout)

iplot(fig)